# 特定数据类型的解析，主要是日期、时间、HTML。

庆幸有几个包可以用，可以避开复杂的正则表达式。它们可作为NLTK的补充：

    dateutil: 提供日期解析和时区转换
    lxml、BeautifulSoup: HTML的解析、清洗、转换
    charade、UnicodeDammit: 检测、转换文本字符编码
    
1. raw文本数据 -——>经这几个包处理后 ——> 才能传入NLTK object
2. 或者NLTK处理后的后期加工，也可用这几个包

## 具体的，假定要解析一篇关于某个餐厅的博客文章。

你可以用 *lxml* 或者 *BeautifulSoup* 提取文章文本、链接、日期和时间。

日期和时间可以通过*dateutil* 解析为 Python datetime object 。

解析出文本后，你可以用 *charade* 确保它是 utf-8，这样之后才能跑通 基于NLTK 的词性标注、chunk提取、文本分类等来获取文章的元数据。

现实中，文本处理绝不仅仅是基于NLTK的操作，通常有更多额外的处理需求。本章会覆盖这些内容。

## Parsing Dates & Times with Dateutil

deteutil 是解析日期、时间最好的包，无出其右。

### parser.parse( datetime string )  能解析比这里展示的多得多的日期格式

只能解析 datetime string，若是string 会报错

In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [22]:
from dateutil import parser
parser.parse('Thu Sep 25 10:36:28 2010') # 返回的object是 Python的datetime.datetime

datetime.datetime(2010, 9, 25, 10, 36, 28)

In [2]:
parser.parse('Thursday,25,September 2010 10:36AM')

datetime.datetime(2010, 9, 25, 10, 36)

In [3]:
parser.parse('9/25/2010 10:36:28')

datetime.datetime(2010, 9, 25, 10, 36, 28)

In [4]:
parser.parse('9/25/2010')

datetime.datetime(2010, 9, 25, 0, 0)

In [5]:
parser.parse('2010-09-25T10:26:28Z')

datetime.datetime(2010, 9, 25, 10, 26, 28, tzinfo=tzutc())

无需正则表达式，它能辨识日期tokens并尽可能猜出其所指的日期。

至于不同文化惯用的日期格式，比如月／日／年（dateutil默认格式），日／月／年等，可通过参数***dayfirst***设置

In [15]:
parser.parse('7/8/2017') 
parser.parse('7/8/2017', dayfirst=True) # 7日

datetime.datetime(2017, 7, 8, 0, 0)

datetime.datetime(2017, 8, 7, 0, 0)

In [17]:
parser.parse('10-9-25')  # 默认 月／日／年
parser.parse('10-9-8', yearfirst=True)  # 年／月／日

datetime.datetime(2025, 10, 9, 0, 0)

datetime.datetime(2010, 9, 8, 0, 0)

### fuzzy parser

deteutil parser 也能做模糊解析，可以忽略 非datetime string 的无关字符。只需设置 ***fuzzy=Ture***

In [18]:
try:
    parser.parse('9/25/2010 at about 10:36AM')
except ValueError:
    'cannot parse'
    
parser.parse('9/25/2010 at about 10:36AM in Beijing', fuzzy=True)

'cannot parse'

datetime.datetime(2010, 9, 25, 10, 36)

## 时区的查找和转换

大部分datetime 对象是naive 的， 不带有具体时区信息，即时区和UTC offset。

UTC: Uiversal Time Coordinated 世界调整时，也就是GMT 格林尼治时间 Greenwich Mean Time。

ISO  International Standards Organization 国际标准化组织，指定标准的datetime格式。

Python datetime对象既可以是naive ，也可以aware。

要使得naive 变得aware，必须给定具体的tzinfo。

In [20]:
from dateutil import tz
tz.tzutc()  # a UTC tzinfo object

tzutc()

In [26]:
import datetime
datetime.datetime.utcnow() # a UTC datetime object:
tz.tzutc().utcoffset(datetime.datetime.utcnow()) # check that the offset is 0 

datetime.datetime(2017, 8, 18, 6, 23, 11, 27126)

datetime.timedelta(0)

In [23]:
tz.gettz('US/Pacific') # To get 其他时区的tzinfo objects，传入tz file的路径
tz.gettz('US/Pacific').utcoffset(datetime.datetime.utcnow())

tzfile('/usr/share/zoneinfo/US/Pacific')

datetime.timedelta(-1, 61200)

可见：UTC offsets 是datetime.timedelta 对象,第一个数是（时差的）days,第二个数是seconds

In [24]:
tz.gettz('Europe/Paris')
tz.gettz('Europe/Paris').utcoffset(datetime.datetime.utcnow())

tzfile('/usr/share/zoneinfo/Europe/Paris')

datetime.timedelta(0, 7200)

若要将datetimes存入数据库，建议全以UTC形式，消除时区歧义。即是数据库能自动辨识时区。

## To convert a non-UTC datetime object to UTC，必须指明时区。
否则，将一个naive datetime 转换为UTC 会报错。

对naive datetime 指明时区，只需调用***replace()*** with the correct ***tzinfo*** 。

datetime object 有了***tzinfo*** 之后，只需调用 ***astimezone()*** with ***tz.tzutc()***

In [12]:
from dateutil import tz
import datetime
pst = tz.gettz('US/Pacific')
dt = datetime.datetime(2010, 9, 25, 10, 36)

dt   # naive datetime
print(dt)
print(dt.tzinfo)

datetime.datetime(2010, 9, 25, 10, 36)

2010-09-25 10:36:00
None


In [16]:
# naive 情况下转换，现版本不会报错了
dt.astimezone(tz.tzutc())  
print(dt.astimezone(tz.tzutc()))

datetime.datetime(2010, 9, 25, 2, 36, tzinfo=tzutc())

2010-09-25 02:36:00+00:00


In [20]:
dt.replace(tzinfo=pst)   # aware
print(dt.replace(tzinfo=pst).tzinfo)
print(dt.replace(tzinfo=pst))
print(dt.replace(tzinfo=pst).astimezone(tz.tzutc()))

datetime.datetime(2010, 9, 25, 10, 36, tzinfo=tzfile('/usr/share/zoneinfo/US/Pacific'))

tzfile('/usr/share/zoneinfo/US/Pacific')
2010-09-25 10:36:00-07:00
2010-09-25 17:36:00+00:00


replace() 和 astimezone() 均返回新的object，不改动当前object

### parser解析时,存在无法检出的时区信息（但确实能作为tzinfo），那么传入参数tzinfos

In [23]:
parser.parse('Wednesday, Aug 4, 2010 at 6:30 p.m. (CDT)', fuzzy=True) # naive datetime

tzinfos = {'CDT': tz.gettz('US/Central')} #传入到参数tzinfos，有所映射，就能识别naive中未识别的tzinfo
parser.parse('Wednesday, Aug 4, 2010 at 6:30 p.m. (CDT)', fuzzy=True, tzinfos=tzinfos) # aware

datetime.datetime(2010, 8, 4, 18, 30)

datetime.datetime(2010, 8, 4, 18, 30, tzinfo=tzfile('/usr/share/zoneinfo/US/Central'))

tzoffset('custom', 3600)

### Custom offsets 自定义偏移
创建自己的tzinfo object，偏移1小时：

In [27]:
tz.tzoffset('custom', 3600) # 第一个参数是对象名； 1 hour = 3600s

tzoffset('custom', 3600)

## 用 lxml 从HTML网页中提取 URLs链接

lxml 有一个 html module，专门用于解析HTML。

可用 ***.fromstring()*** 函数解析HTML

***.iterlinks()*** 函数,return 生成器。生成 4-tuples (element, attr, link, pos):

    element: 解析节点的anchor tag，links从中提取；若只对link 感兴趣，可忽略
    attr: 链接属性，通常为 href
    link: 提取的URL
    pos: document中，锚标签的数值index，第一个tag的pos为0，第二个为1，以此类推

In [1]:
from lxml import html
doc = html.fromstring('Hello <a href="/world">world</a>') #解析HTML
links = list(doc.iterlinks()) # 解析后得到元组生成器，list得到 所有links 的列表
len(links)

1

In [4]:
links

[(<Element a at 0x10962b728>, 'href', '/world', 0)]

In [3]:
(el, attr, link, pos) = links[0]
attr

'href'

In [5]:
link

'/world'

In [11]:
pos

0

In [6]:
doc.make_links_absolute('http://hello')
abslinks = list(doc.iterlinks())
(el, attr, link, pos) = abslinks[0]
link

'http://hello/world'

In [7]:
links = list(html.iterlinks('Hello <a href="/world">world</a>'))
links[0][2]

'/world'

In [8]:
doc.xpath('//a/@href')[0]

'http://hello/world'